### Ячейка с библиотеками:

In [1]:
import pandas as pd
import re
import nltk
import pymorphy2

C:\Users\feygi\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
data = pd.read_csv('test_data.csv')

### Задача №1: извлекать реплики с приветствием – где менеджер поздоровался.

Используем для этого самый тривиальный способ:

1) вычленим из таблицы все реплики менеджера;
2) составим список возможных приветствий (hellos);
3) создадим список, в который мы вложим полученные реплики с приветствиями (list1);
4) через циклы проверим, в каких строчках присутствуют ключевые слова из hellos. 
5) собираем нужные реплики в список list1.

In [3]:
#заводим список, содержащий все реплики менеджеров
manager_words = data[data['role'] == 'manager']
#хорошенько подумав, ограничиваем список до первых нескольких реплик, в которые и должно входить приветствие
m_words = manager_words[manager_words['line_n'] <= 5]
mw = m_words['text']
#список слов, употребляющихся при приветствии
hellos = ['здравствуйте', 'добрый', 'Здравствуйте', 'Добрый']
#итоговый список с искомыми репликами
list1 = []
def check(spis_replic, spis_dop_slov, itog):
    for i in spis_replic:
        for k in spis_dop_slov:
            if k in i:
                itog.append(i)
    return itog
check(mw, hellos, list1)

['Алло здравствуйте',
 'Алло здравствуйте',
 'Алло здравствуйте',
 'Алло дмитрий добрый день',
 'Добрый меня максим зовут компания китобизнес удобно говорить']

### Задача №2: извлекать имя менеджера. 

Найдём возможные имена с помощью библиотек pymorphy2 и nltk, а также циклов

In [4]:
#рассматривались разные величины, начиная с 0.1 до 0.9. Была выбрана 0.2, ибо только в этом случае появляется "максим"
prob_thresh = 0.2 

morph = pymorphy2.MorphAnalyzer()
#список возможных менеджеров
probable_manager = []
for m in mw:
    for word in nltk.word_tokenize(m):
        for pr in morph.parse(word):
            if 'Name' in pr.tag and pr.score >= prob_thresh:
                probable_manager.append(word)
                print('{:<12}\tвероятность:\t{:0.3}'.format(word, pr.score))

ангелина    	вероятность:	1.0
ангелина    	вероятность:	1.0
ангелина    	вероятность:	1.0
дмитрий     	вероятность:	0.985
максим      	вероятность:	0.25
дмитрий     	вероятность:	0.985
анастасия   	вероятность:	0.333
анастасия   	вероятность:	0.333
анастасия   	вероятность:	0.333
анастасия   	вероятность:	0.333
анастасия   	вероятность:	0.333
анастасия   	вероятность:	0.333


In [5]:
#в этот словарь мы складываем предложения, содержащие имена менеджеров. И избавляемся от повторов
managers = []
check(mw, probable_manager, managers)
managers = list(set(managers))

In [6]:
#заводим список слов, как правило, фигурирующих в приветствиях
exceptions = ['Меня', 'меня', 'это', 'Это', 'зовут']
#в список ниже складываются предложения из предыдущего, содержащие слова из exceptions
man_names = []
check(managers, exceptions, man_names)
man_names = list(set(man_names))

In [7]:
#сюда поступает конечный результат - имена менеджеров.
definite_managers = []
for mn in man_names:
    for w in nltk.word_tokenize(mn):
        for p in morph.parse(w):
            if 'Name' in p.tag and p.score >= prob_thresh:
                definite_managers.append(w)
                definite_managers = list(set(definite_managers))
definite_managers

['анастасия', 'ангелина', 'максим']

### Задача № 3: извлекать реплики, где менеджер представил себя.

Из прошлых заданий мы получили  список предложений со словами, которые фигурируют в представлениях менеджерами самих себя. Таким образом, у нас есть список man_names, однако не все входящие в него реплики можно назвать подходящими под заданное условие. Потому следует задаться вопросом: что именно стоит называть представлением? Полагаю, что представление должно состоять из имени менеджера и компании, которую он представляет. Поэтому необходимо включить в "фильтр" слово "компания" и производные от него.

In [8]:
companies = ['компания', 'компанию', 'компании']
present = []
check(man_names, companies, present)
present = list(set(present))

### Задача №4: извлекать название компании.

Из прошлого задания мы получили список представлений менеджеров. В нём, как мы видим, можно заметить и название компаний. Наша задача - извлечь их.

In [9]:
comp = []
for p in present:
    pp = p.find('компания')
    end = p.find('бизнес')
    name = p[pp+0:end+6]
    comp.append(name)
comp

['компания китобизнес',
 'компания диджитал бизнес',
 'компания диджитал бизнес',
 'компания диджитал бизнес']

### Задача №5: извлекать реплики, где менеджер попрощался.

В данном случае воспользуемся библиотекой re:

In [10]:
managerw = manager_words['text']
for r in managerw:
    #настроим поиск на слова, чаще всего употребляющиеся в качестве прощальных в деловой речи
    if re.search('свидания', r) or re.search('доброго', r): 
        print(r)

Всего хорошего до свидания
До свидания
Угу все хорошо да понедельника тогда всего доброго
Во вторник все ну с вами да тогда до вторника до свидания
Ну до свидания хорошего вечера


### Задача №6: проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [11]:
#найдём количество диалогов
num_dialogues = len(manager_words['dlg_id'].value_counts())
#создадим функцию для создания таблиц, чтобы в последствии сравнить одну из их колонок
def df_creator(dialogues, basic_tab, words):
    numr = []
    replicas = []
    for num in range(dialogues):
        num_words = basic_tab[basic_tab['dlg_id'] == num]
        text_num_words = num_words['text']
        for replica in text_num_words:
            for h in words:
                if re.search(h, replica):
                    numr.append(num)
                    replicas.append(replica)
                    tabl = pd.DataFrame({'num':numr, 'greetings|byes':replicas})
    return tabl

df1 = df_creator(num_dialogues, manager_words, hellos)
df2 = df_creator(num_dialogues, manager_words, ['свидания', 'доброго'])
done = list(set(df1['num']) & set(df2['num']))
for x in done:
    print('В диалоге номер '+str(x)+' оба условия соблюдены.')
not_done = list(set(df1['num']) ^ set(df2['num']))
for y in not_done:
    print('В диалоге номер '+str(y)+' не соблюдено одно или более условий.')


В диалоге номер 0 оба условия соблюдены.
В диалоге номер 1 оба условия соблюдены.
В диалоге номер 3 оба условия соблюдены.
В диалоге номер 2 не соблюдено одно или более условий.
В диалоге номер 4 не соблюдено одно или более условий.
В диалоге номер 5 не соблюдено одно или более условий.
